# 데이터 수집 - 네이버 영화 정보(리뷰수, 배우)

### * 데이터 불러오기

In [ ]:
import pandas as pd

In [ ]:
# 데이터 불러오기
train = pd.read_csv('영화관객수/movies_train.csv')
test = pd.read_csv('영화관객수/movies_test.csv')

In [ ]:
# 제목 확인 
train_title = list(train.title)
test_title = list(test.title)

### * 필요 라이브러리
- BeautifulSoup
   : HTML정보로 부터 원하는 데이터를 가져오기 쉽게,비슷한 분류의 데이터별로 나누어주는(parsing) 파이썬 라이브러리
- selenium
   : 다양한 브라우저 및 플랫폼에서 웹 응용프로그램을 위한 자동화 테스트를 도와주는 무료 툴. 

### * Chromdriver
- chorme브라우저 -> 도움말 > Chrome정보 -> 크롬버전확인
- https://chromedriver.chromium.org/downloads 에서 버전에 맞는 크롬브라우저 다운로드
- chromedriver 파일 위치 설정

In [ ]:
# BeautifulSoup4 설치
#! pip install bs4

In [ ]:
# selenium 설치
#!pip install selenium

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import datetime
import re

In [ ]:
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(1)
driver.get('https://movie.naver.com/')
time.sleep(1)

<ipython-input-127-fcb55c25a7e6>:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


### -------- Crawling TEST ---------

In [ ]:
# 검색창 찾기
search_input = driver.find_element(By.ID, 'ipt_tx_srch')

<selenium.webdriver.remote.webelement.WebElement (session="9a26f7a88dde033dd9b96b503cf1375a", element="91585d60-d22f-473c-a8ff-bdfbb72f8901")>


In [ ]:
search_input.clear()
search_input.send_keys("길위에서")

In [ ]:
# 영화 검색 
search_submit = driver.find_element(By.XPATH, '//div[@class="srch_field_on _view"]/button')
search_submit.click()
time.sleep(1)

In [ ]:
# 영화 검색 후 접근 완료
selected_movie = driver.find_element(By.XPATH, '//ul[@class="search_list_1"]/li/p/a')
selected_movie.click()
time.sleep(1)

In [ ]:
# 리뷰 클릭
review = driver.find_element(By.XPATH, '//div[@class="sub_tab_area"]/ul/li[last()-1]') 
review.click()
time.sleep(1)

In [ ]:
# 영화 정보 크롤링 BeautifulSoup로 html정보 분석
html = driver.page_source

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
title = soup.select_one(".h_movie > a").text

In [ ]:
title

'길 위에서'

In [ ]:
actors_a_el = soup.select("dl.info_spec > dd:nth-child(6) > p > a")
actors_list = [actor.text for actor in actors_a_el]
actors = "|".join(actors_list)
actors

''

In [ ]:
try:
    # `title` 영화 제목 
    title = soup.select_one(".h_movie > a").text
    print(title)
    
    # `actors` 배우 정보 (구분자: "|")
    actors_a_el = soup.select("dl.info_spec > dd:nth-child(6) > p > a")
    actors_list = [actor.text for actor in actors_a_el]
    actors = "|".join(actors_list)
    print(actors)
    
    # `review_cnt` 네이버 리뷰수 추가
    review_cnt = int(soup.select_one(".top_behavior .cnt em").text.replace(",",""))
    print(review_cnt)

    # `naver_director` 감독정보
    director = soup.select_one("dl.info_spec > dd:nth-child(4) > p > a").text
    print(director)
except: 
    print("error")

### -------- Crawling TEST END ---------
<hr>

## 1. train set 크롤링

In [ ]:
len(train_title)

600

In [ ]:
# 반복문
train_add = []
for title in train_title:
    info = [title]
    title = re.sub("[^\w\s]", "", title) # 특수문자 제거
    search_title = re.sub(r"\s", "", title) # 공백제거
    info.append(search_title)
    
    # 검색창에 input 넣기
    search_input = driver.find_element(By.ID, 'ipt_tx_srch')
    search_input.clear()
    search_input.send_keys(search_title)
    
    # 영화 검색 클릭
    search_submit = driver.find_element(By.XPATH, '//div[@class="srch_field_on _view"]/button')
    search_submit.click()
    time.sleep(1)
    
    # 검색결고 있는지 확인
    try: 
        # 영화 검색 후 상세 페이지 접근 
        selected_movie = driver.find_element(By.XPATH, '//ul[@class="search_list_1"]/li/p/a')
        selected_movie.click()
        time.sleep(1)

        # 리뷰 클릭
        review = driver.find_element(By.XPATH, '//div[@class="sub_tab_area"]/ul/li[last()-1]') 
        review.click()
        time.sleep(0.5)

        # 영화 정보 크롤링 BeautifulSoup로 html정보 분석
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # `title` 영화 제목 
        try: 
            title = soup.select_one(".h_movie > a").text
            info.append(title)
        except: 
            info.append("error")

        # `naver_director` 영화 제목 
        try: 
            director = soup.select_one("dl.info_spec > dd:nth-child(4) > p > a").text
            info.append(director)
        except: 
            info.append("error")

        # `actors` 배우 정보 (구분자: "|")
        try: 
            actors_a_el = soup.select("dl.info_spec > dd:nth-child(6) > p > a")
            actors_list = [actor.text for actor in actors_a_el]
            actors = "|".join(actors_list)
            info.append(actors)
        except: 
            info.append("error")


        # 네이버 리뷰수 추가
        try: 
            review_cnt = int(soup.select_one(".top_behavior .cnt em").text.replace(",",""))
            info.append(review_cnt)
        except: 
            info.append("error")

        # 추가된 정보 확인
        print(info)
        train_add.append(info)
        
    # 검색결과 없을 경우 error출력
    except:
        info = [title, search_title, "error", "error", "error", "error"]
        print(train_add.append(info))
    

In [ ]:
len(train_add)

600

In [ ]:
# 크롤링한 데이터 dataframe화
train_add_df = pd.DataFrame(train_add, columns = ['title', 'search_title','naver_title','naver_director', 'actors','review_cnt'] )
train_add_df.head(3)

,title,search_title,naver_title,naver_director,actors,review_cnt
0,개들의 전쟁,개들의전쟁,개들의 전쟁,조병옥,김무열|진선규,134
1,내부자들,내부자들,내부자들,우민호,이병헌|조승우|백윤식,2779
2,은밀하게 위대하게,은밀하게위대하게,은밀하게 위대하게,장철수,김수현|박기웅|이현우,3028


In [ ]:
# 기존 데이터와 병합 후, `result` 컬럼 값을 통해 잘못 들어간 정보 확인
train_mearge = pd.merge(train,train_add_df,how='left')
train_mearge['result'] = train_mearge['director'] == train_mearge['naver_director']
train_mearge.head(3)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,search_title,naver_title,naver_director,actors,review_cnt,result
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398,개들의전쟁,개들의 전쟁,조병옥,김무열|진선규,134,True
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501,내부자들,내부자들,우민호,이병헌|조승우|백윤식,2779,True
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083,은밀하게위대하게,은밀하게 위대하게,장철수,김수현|박기웅|이현우,3028,True


In [ ]:
# train_mearge_1.csv로 저장
train_mearge.to_csv('train_mearge_1.csv',index=False)

In [ ]:
# 600개 중 99개 데이터가 잘못 크롤링됨
len(train_mearge[train_mearge['result']==False])

99

### train set 크롤링 결과
- 크롤링 결과 파일명 : **'train_merge_1.csv'**
- len : 600 rows
- 데이터 검증
    - 동일한 제목의 영화 데이터로 잘못 얻어진 경우 존재 -> 제목은 같으나, 감독이름이 다른 경우로 판단
    - 600개 중 501개 정상(제목, 감독 동일), 99개(감독 다름) 
    - 99개 수기로 검색 후 수정 -> train_merge_2
- 크롤링 최종 수정 결과 파일명 : **'train_merge_2.csv'**


<hr>

## 2. test set 크롤링

In [ ]:
# 반복문
test_add = []
for title in test_title:
    info = [title]
    title = re.sub("[^\w\s]", "", title) # 특수문자 제거
    search_title = re.sub(r"\s", "", title) # 공백제거
    info.append(search_title)
    
    # 검색창에 input 넣기
    search_input = driver.find_element(By.ID, 'ipt_tx_srch')
    search_input.clear()
    search_input.send_keys(search_title)
    
    # 영화 검색 클릭
    search_submit = driver.find_element(By.XPATH, '//div[@class="srch_field_on _view"]/button')
    search_submit.click()
    time.sleep(1)
    
    # 검색결고 있는지 확인
    try: 
        # 영화 검색 후 상세 페이지 접근 
        selected_movie = driver.find_element(By.XPATH, '//ul[@class="search_list_1"]/li/p/a')
        selected_movie.click()
        time.sleep(1)

        # 리뷰 클릭
        review = driver.find_element(By.XPATH, '//div[@class="sub_tab_area"]/ul/li[last()-1]') 
        review.click()
        time.sleep(0.5)

        # 영화 정보 크롤링 BeautifulSoup로 html정보 분석
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # `title` 영화 제목 
        try: 
            title = soup.select_one(".h_movie > a").text
            info.append(title)
        except: 
            info.append("error")

        # `naver_director` 영화 제목 
        try: 
            director = soup.select_one("dl.info_spec > dd:nth-child(4) > p > a").text
            info.append(director)
        except: 
            info.append("error")

        # `actors` 배우 정보 (구분자: "|")
        try: 
            actors_a_el = soup.select("dl.info_spec > dd:nth-child(6) > p > a")
            actors_list = [actor.text for actor in actors_a_el]
            actors = "|".join(actors_list)
            info.append(actors)
        except: 
            info.append("error")


        # 네이버 리뷰수 추가
        try: 
            review_cnt = int(soup.select_one(".top_behavior .cnt em").text.replace(",",""))
            info.append(review_cnt)
        except: 
            info.append("error")

        # 추가된 정보 확인
        #print(info)
        test_add.append(info)
        
    # 검색결과 없을 경우 error출력
    except:
        info = [title, search_title, "error", "error", "error", "error"]
        #print(info)
        test_add.append(info)
        
    

In [ ]:
# 크롤링한 데이터 dataframe화
test_add_df = pd.DataFrame(test_add, columns = ['title', 'search_title','naver_title','naver_director', 'actors','review_cnt'] )
test_add_df.head(3)

,title,search_title,naver_title,naver_director,actors,review_cnt
0,용서는 없다,용서는없다,용서는 없다,김형준,설경구|류승범|한혜진,536
1,아빠가 여자를 좋아해,아빠가여자를좋아해,아빠가 여자를 좋아해,이광재,이나영|김지석|김희수,221
2,하모니,하모니,하모니,셀린 가유르드,,0


In [ ]:
# 기존 데이터와 병합 후, `result` 컬럼 값을 통해 잘못 들어간 정보 확인
test_mearge = pd.merge(test,test_add_df,how='left')
test_mearge['result'] = test_mearge['director'] == test_mearge['naver_director']
test_mearge.head(3)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,search_title,naver_title,naver_director,actors,review_cnt,result
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,3.005290e+05,2,304,3,용서는없다,용서는 없다,김형준,설경구|류승범|한혜진,536,True
1,아빠가 여자를 좋아해,(주)쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,3.427002e+05,4,275,3,아빠가여자를좋아해,아빠가 여자를 좋아해,이광재,이나영|김지석|김희수,221,True
2,하모니,CJ 엔터테인먼트,드라마,2010-01-28,115,12세 관람가,강대규,4.206611e+06,3,419,7,하모니,하모니,셀린 가유르드,,0,False


In [ ]:
# test_mearge_1.csv로 저장
test_mearge.to_csv('test_merge_1.csv',index=False)

In [ ]:
# 243개 중 51개 데이터가 잘못 크롤링됨
len(test_mearge[test_mearge['result']==False])

51

In [ ]:
len(test_title), len(test_add)

(243, 243)

### test set 크롤링 결과
- 크롤링 결과 파일명 : **'test_merge_1.csv'**
- len : 243 rows
- 데이터 검증
    - 동일한 제목의 영화 데이터로 잘못 얻어진 경우 존재 -> 제목은 같으나, 감독이름이 다른 경우로 판단
    - 243개 중 182개 정상(제목, 감독 동일), 51개(감독 다름) 
    - 51개 수기로 검색 후 수정 -> test_merge_2.csv
- 크롤링 최종 수정 결과 파일명 : **'test_merge_2.csv'**

<hr>

## 3. 크롤링 결과
### * 파일
- train 데이터 크롤링 결과 파일 : **train_merge_2.csv**
- test 데이터 크롤링 결과 파일 : **test_merge_2.csv**

### * 추가된 변수
- **actors** : 네이버 영화에 나오는 배우 정보
- **review_cnt** : 네이버 영화 상세 페이지의 리뷰 개수
